## Analyse af lager vs realisationsbeskatning ved forskellige depotstørrelser

In [39]:
import os
import sys
module_path = os.path.abspath(os.path.join('/home/jovyan/work/dkfinance_modeller'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [40]:
import os
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import dkfinance_modeller.aktieskat.depotmodel as depotmodel
import dkfinance_modeller.aktieskat.kurtage as kurtage
import dkfinance_modeller.aktieskat.skat as skat
import dkfinance_modeller.aktieskat.vaerdipapirer as værdipapirer
import dkfinance_modeller.aktieskat.valuta as valuta
import dkfinance_modeller.simple_modeller.formler as formler

Opsætning af depoter

In [41]:
def depoter(kapital : float) -> Tuple[depotmodel.DepotModel, depotmodel.DepotModel]:
    """Definere depoter.

    Returns:
      Depot med realationsbeskatning
    """
    etf1 = værdipapirer.ETF(kurs=100, åop=0.12 / 100, beskatningstype='lager')
    etf2 = værdipapirer.ETF(kurs=100, åop=0.12 / 100, beskatningstype='lager')
    skatter1 = skat.Skat(beskatningstype='aktie')
    skatter2 = skat.Skat(beskatningstype='pension')
    lagerbeskatning = depotmodel.DepotModel(
        kapital=kapital,
        kurtagefunktion=kurtage.saxo_kurtage_bygger(valuta="eur", valutakurs=7.44, underkonto=True),
        skatteklasse=skatter1,
        minimumskøb=5000,
        ETFer=[etf1],
        ETF_fordeling=[1.0],
        valutafunktion=valuta.saxo_underkonto_kurtage,
    )
    realisationsbeskatning = depotmodel.DepotModel(
        kapital=kapital,
        kurtagefunktion=kurtage.saxo_kurtage_bygger(valuta="eur", valutakurs=7.44, underkonto=True),
        skatteklasse=skatter2,
        minimumskøb=5000,
        ETFer=[etf1],
        ETF_fordeling=[1.0],
        valutafunktion=valuta.saxo_underkonto_kurtage,
    )
    return lagerbeskatning, realisationsbeskatning

In [42]:
def output(dep):
    out = [    
    dep.total_salgsværdi(), 
    dep.ubeskattet, 
    dep._kapital, 
    dep.ETFer[0].antal_værdipapirer, 
    dep.ETFer[0].antal_værdipapirer * dep.ETFer[0].kurs,dep.ETFer[0].kurs, 
    dep.ETFer[0].beskattet_kurs, 
    dep.måned
       
    ]
    return out
outsample = pd.DataFrame(list(str('') * 10), columns = range(0,12)) 

Beregning af slutværdi

In [43]:
data = np.genfromtxt("SP500.csv", delimiter=";")
data[:,3]=0.06/12
data[:,4]=0.02/12


In [44]:
lagerdepot, realisationsdepot = depoter(100000)
i = 1
for x in range(0,11):
                udbytteafkast_real = realisationsdepot.ETFer[0].kurs * data[i + 1, 4]
                kursafkast_real = realisationsdepot.ETFer[0].kurs * data[i + 1, 3]
                udbytteafkast_lager = lagerdepot.ETFer[0].kurs * data[i + 1, 4]
                kursafkast_lager = lagerdepot.ETFer[0].kurs * data[i + 1, 3]
                realisationsdepot.afkast_månedlig([udbytteafkast_real + kursafkast_real], [0.0])
                lagerdepot.afkast_månedlig([kursafkast_lager+udbytteafkast_lager], [0.0])  

a = [lagerdepot, realisationsdepot]

In [45]:
lagerdepot.måned,lagerdepot.total_salgsværdi(), lagerdepot.ETFer[0].total_værdi()

(11, 221336.92764563762, 230169.56404301242)

In [46]:
realisationsdepot.måned,realisationsdepot.total_salgsværdi(), realisationsdepot.ETFer[0].total_værdi()

(11, 224936.96663867007, 230169.56404301242)

In [47]:
realisationsdepot.afkast_månedlig([0.0], [0.0])
lagerdepot.afkast_månedlig([0.0], [0.0])  
lagerdepot.skatteklasse.skatteprocenter, realisationsdepot.skatteklasse.skatteprocenter

([0.27, 0.42], [0.153])

## til Erik: 
meget forskellig skatte % men alligevel samme resultat efter første skatteopgørelse?

In [48]:
lagerdepot.måned, lagerdepot.total_salgsværdi(), lagerdepot.ETFer[0].total_værdi()

(0, 224878.8632072695, 225391.7800196486)

In [49]:
realisationsdepot.måned, realisationsdepot.total_salgsværdi(), realisationsdepot.ETFer[0].total_værdi()

(0, 224923.10261631283, 225391.7800196486)